# Maailmanpankin kehitysindikaattorit

Aja alla olevat solut ja tarkastele interaktiivisen työkalun avulla Maailmanpankin kehitysindikaattoreita kartalla.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Luetaan maailmankartalle tarvittava data geopandasin arkistosta
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Korjataan puuttuvat maakoodit datassa
world.at[43, 'iso_a3'] = "FRA"
world.at[21, 'iso_a3'] = "NOR"
world.at[167, 'iso_a3'] = "SOM"
world.at[174, 'iso_a3'] = "XKX"

# Määritellään indikaattorikategoriat ja datatiedostot, mistä data löytyy
ind_dict = {'Agriculture & Rural Development': "indicator_data/API_1_DS2_en_csv_v2_2169365.csv",
            'Aid Effectiveness': "indicator_data/API_2_DS2_en_csv_v2_2180514.csv",
            'Climate Change': "indicator_data/API_19_DS2_en_csv_v2_2165528.csv",
            'Economy & Growth': "indicator_data/API_3_DS2_en_csv_v2_2169088.csv",
            'Education': "indicator_data/API_4_DS2_en_csv_v2_2165022.csv",
            'Energy & Mining': "indicator_data/API_5_DS2_en_csv_v2_2167659.csv",
            'Environment': "indicator_data/API_6_DS2_en_csv_v2_2179282.csv",
            'External Debt': "indicator_data/API_20_DS2_en_csv_v2_2170727.csv",
            'Financial Sector': "indicator_data/API_7_DS2_en_csv_v2_2180150.csv",
            'Gender': "indicator_data/API_17_DS2_en_csv_v2_2179188.csv",
            'Health': "indicator_data/API_8_DS2_en_csv_v2_2166113.csv",
            'Infrastructure': "indicator_data/API_9_DS2_en_csv_v2_2179261.csv",
            'Poverty': "indicator_data/API_11_DS2_en_csv_v2_2163688.csv",
            'Private Sector': "indicator_data/API_12_DS2_en_csv_v2_2164513.csv",
            'Public Sector': "indicator_data/API_13_DS2_en_csv_v2_2179366.csv",
            'Science & Technology': "indicator_data/API_14_DS2_en_csv_v2_2171746.csv",
            'Social Development': "indicator_data/API_15_DS2_en_csv_v2_2180590.csv",
            'Social Protection & Labor': "indicator_data/API_10_DS2_en_csv_v2_2173106.csv",
            'Trade': "indicator_data/API_21_DS2_en_csv_v2_2163553.csv",
            'Urban Development' : "indicator_data/API_16_DS2_en_csv_v2_2167825.csv" }

In [ ]:
from ipywidgets import interact, Dropdown, IntSlider
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import Greens
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool

min_year = 1960 # ensimmäinen vuosi datassa
max_year = 2020 # viimeisin vuosi datassa
default_year = 2018 # viimeisin vuosi, jossa on paljon dataa 

# Luodaan pudotusvalikot ja liukusäädin
indicator_category = Dropdown(options = ind_dict.keys())
indicator = Dropdown()
years = IntSlider(min=min_year, max=max_year, continuous_update=False, value=default_year)

# Funktio, joka lukee datan indikaattorikategorian perusteella valitusta tiedostosta
# sekä asettaa datan sisältämät indikaattorit toiseen pudotusvalikkoon.
# Tätä kutsutaan, kun kategoriavalikosta valitaan uusi kategoria
@interact(Category = indicator_category)
def choose_indicator_group(Category):
    filepath = ind_dict[Category]
    data = pd.read_csv(filepath, skiprows=4)
    merged_data = world.merge(data, left_on='iso_a3', right_on='Country Code')
    
    # Muokataan globaalia muuttujaa, jotta sitä voidaan käyttää seuraavassa funktiossa
    global indicator_groups
    indicator_groups = merged_data.groupby("Indicator Name")
    indicators = list(indicator_groups.groups.keys())
    indicator.options = indicators
    
# Funktio, joka piirtää kartan valitusta indikaattorista valittuna vuotena
@interact(Indicator = indicator, Year = years)
def f(Indicator, Year):
    group_data = indicator_groups.get_group(Indicator)
    yearly_data = group_data[str(Year)]

    # Jos dataa ei löydy, tulostetaan ainoastaan "No data".
    # Karttaa ei voi tulostaa kokonaan harmaana, koska Bokeh ei salli sitä, että
    # datan kaikki pisteet ovat "NaN"-muotoa
    if yearly_data.isnull().all():
        print("No data")
        return
    
    data_json = group_data.to_json()
    geosource = GeoJSONDataSource(geojson = data_json)
    palette = Greens[6][::-1]
    color_mapper = LinearColorMapper(palette = palette,
                                     low = yearly_data.min(),
                                     high = yearly_data.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)
    fig = figure(title=Indicator, width=900, height=550)
    fig.add_layout(color_bar, 'right')
    fig.patches('xs','ys',
                source = geosource,
                fill_color={'field':str(Year),'transform': color_mapper})
    hover = HoverTool()
    hover.tooltips = [('Country', '@name'),('Value', f'@{Year}')]
    fig.add_tools(hover)

    output_notebook()
    show(fig)